In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import warnings
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset




DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data_folder = os.path.join(os.curdir, 'AllData')
data_path = os.path.join(data_folder, 'weather.tsv')
df = pd.read_csv(data_path, sep='\t')  

df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.dtypes

In [ ]:
df['time'] = pd.to_datetime(df['time'])

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
plt.hist(df['wind_direction'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Wind Speed')
plt.ylabel('Frequency')
plt.title('Wind Direction Distribution')
plt.show()

In [ ]:
# Here we notice that wind direction have values more than 360 degress which is not possible
# I think it is a mistake in the data so we will replace it with the same value subtracted by 360 because most of the wind direction comes from the north in Jordan as we saw in the plot above
df.loc[df['wind_direction'] > 360, 'wind_direction'] = df['wind_direction'] - 360

In [ ]:
plt.hist(df['temperature'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.title('Temperature Distribution')
plt.show()

In [ ]:
plt.hist(df['wind_speed'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Wind Speed')
plt.ylabel('Frequency')
plt.title('Wind Speed Distribution')
plt.show()

In [ ]:
plt.hist(df['wind_direction'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Wind Speed')
plt.ylabel('Frequency')
plt.title('Wind Direction Distribution')
plt.show()

In [ ]:
plt.hist(df['dew_point'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Dew Point')
plt.ylabel('Frequency')
plt.title('Dew Point Distribution')
plt.show()

In [ ]:
plt.hist(df['visibility'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Visibility')
plt.ylabel('Frequency')
plt.title('Visibility Distribution')
plt.show()

In [ ]:
plt.hist(df['clouds.total_cover'], bins=20, color='blue', alpha=0.7)
plt.xlabel('Coulds Total Cover')
plt.ylabel('Frequency')
plt.title('Coulds Total Cover Distribution')
plt.show()

In [ ]:
sns.boxplot(x='dew_point', data=df, color='green')
plt.xlabel('Dew Point')
plt.title('Dew Point Box Plot')
plt.show()

In [ ]:
sns.boxplot(x='temperature', data=df, color='green')
plt.xlabel('Temperature')
plt.title('Temperature Box Plot')
plt.show()

In [ ]:
sns.boxplot(x='wind_speed', data=df, color='green')
plt.xlabel('Wind Speed')
plt.title('Wind Speed Box Plot')
plt.show()

In [ ]:
sns.boxplot(x='wind_direction', data=df, color='green')
plt.xlabel('Wind Direction')
plt.title('Wind Direction Box Plot')
plt.show()

In [ ]:
sns.boxplot(x='visibility', data=df, color='green')
plt.xlabel('Visibility')
plt.title('Visibility Box Plot')    
plt.show()

In [ ]:
sns.boxplot(x='clouds.total_cover', data=df, color='green')
plt.xlabel('Clouds Total Cover')
plt.title('Clouds Total Cover Box Plot')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['time'], df['temperature'], label='Temperature', color='red')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Temperature Time Series')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['time'], df['temperature'], label='Temperature', color='red')
plt.plot(df['time'], df['dew_point'], label='Dew Point', color='blue')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Temperature and Dew Point Time Series')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# calculate correlation matrix and plot it
corr = df.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            cmap='Blues')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# find out the highest correlation
corr.abs().unstack().sort_values(ascending=False).drop_duplicates()

In [ ]:
lags = range(1, 25)  # Change this range as needed
autocorrelation_values = [df['temperature'].autocorr(lag=lag) for lag in lags]

# Display the autocorrelation values for different lags
for lag, acf in zip(lags, autocorrelation_values):
    print(f'Lag {lag}: Autocorrelation = {acf}')

# Plot the autocorrelation values
plt.figure(figsize=(12, 6))
plt.bar(lags, autocorrelation_values)
plt.xlabel('Lag (Hours)')
plt.ylabel('Auto-Correlation')
plt.title('Auto-Correlation of Temperature at Different Lags')
plt.grid(True)

In [ ]:
# Since the data is time-series, we need to split it based on time
split_ratio = 0.8
split_index = int(split_ratio * len(df))

# Split the data into training and testing sets
train_data = df[:split_index]
test_data = df[split_index:]

# Print the sizes of the training and testing sets
print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")

# Dealing with missing data here

In [ ]:
# temperature: Filling missing values with the mean temperature value for the same time of day from nearby days (e.g., the same hour on different days).
# dew_point: Filling missing values with the mean dew point value for the same time of day from nearby days (e.g., the same hour on different days).
# visibility: Filling missing values with the mean visibility value for the same time of day from nearby days (e.g., the same hour on different days).

def fill_missing_with_mean_same_time_neaby_days(df, time_column, column, window_size=7):
    """
    Fill missing values with the mean  for the same time of day from nearby days.

    Parameters:
    - df: DataFrame containing time series data with columns 'time' and 'temperature'.
    - time_column: Name of the column containing timestamps.
    - temperature_column: Name of the column containing temperature values.
    - window_size: Number of days to consider for calculating the mean temperature. Default is 7 days (1 week).

    Returns:
    - DataFrame with missing temperature values filled with the mean temperature.
    """

    # Sort the DataFrame by the 'time' column if not already sorted
    df.sort_values(by=time_column, inplace=True)

    # Create a new DataFrame to store the filled values
    filled_df = df.copy()

    # Iterate through rows with missing temperature values
    for index, row in filled_df.iterrows():
        if pd.isna(row[column]):
            # Get the timestamp and hour of the missing value
            timestamp = row[time_column]
            hour = timestamp.hour

            # Calculate the time window for mean calculation
            start_time = timestamp - pd.DateOffset(days=window_size)
            end_time = timestamp + pd.DateOffset(days=window_size)

            # Filter the DataFrame to select data within the time window
            selected_data = df[(df[time_column] >= start_time) & (df[time_column] <= end_time)]

            # Calculate the mean temperature for the same hour of the day
            mean_temperature = selected_data[selected_data[time_column].dt.hour == hour][column].mean()

            # Fill the missing temperature value with the mean temperature
            filled_df.at[index, column] = mean_temperature

    return filled_df

In [ ]:
# Wind speed: since wind speed contains only 2 missing values, we will fill them with the mean wind speed

def fill_missing_wind_speed_values(df):
    """
    Fill missing wind speed values with the mean wind speed.

    Returns:
    - DataFrame with missing wind speed values filled with the mean wind speed.
    """

    # Fill missing wind speed values with the mean wind speed
    df['wind_speed'].fillna(df['wind_speed'].mean(), inplace=True)

    return df

In [ ]:
# Wind direction: wind direction have significant number of missing values, so we will fill them with the mode 

def fill_missing_wind_direction_values(df):
    """
    Fill missing wind direction values with the mode wind direction.

    Returns:
    - DataFrame with missing wind direction values filled with the mode wind direction.
    """

    # Fill missing wind direction values with the mode wind direction
    df['wind_direction'].fillna(df['wind_direction'].mode()[0], inplace=True)

    return df

In [ ]:
# Fill missing values in the training data and testing data
train_data = fill_missing_with_mean_same_time_neaby_days(train_data, 'time', 'temperature')
train_data = fill_missing_with_mean_same_time_neaby_days(train_data, 'time', 'dew_point')
train_data = fill_missing_with_mean_same_time_neaby_days(train_data, 'time', 'visibility')
train_data = fill_missing_wind_speed_values(train_data)
train_data = fill_missing_wind_direction_values(train_data)

test_data = fill_missing_with_mean_same_time_neaby_days(test_data, 'time', 'temperature')
test_data = fill_missing_with_mean_same_time_neaby_days(test_data, 'time', 'dew_point')
test_data = fill_missing_with_mean_same_time_neaby_days(test_data, 'time', 'visibility')
test_data = fill_missing_wind_speed_values(test_data)
test_data = fill_missing_wind_direction_values(test_data)

In [ ]:
train_data.isna().sum()

In [ ]:
test_data.isna().sum()

In [ ]:
# calculate humidity
train_data['relative_humidity'] = 100 * (np.exp((17.625 * train_data['dew_point']) / (243.04 + train_data['dew_point'])) / np.exp((17.625 * train_data['temperature']) / (243.04 + train_data['temperature'])))
test_data['relative_humidity'] = 100 * (np.exp((17.625 * test_data['dew_point']) / (243.04 + test_data['dew_point'])) / np.exp((17.625 * test_data['temperature']) / (243.04 + test_data['temperature'])))

In [ ]:
corr = train_data.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            cmap='Blues')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# find out the highest correlation
corr.abs().unstack().sort_values(ascending=False).drop_duplicates()

In [ ]:
def lag_feature(df, lag_intervals, column):
    """
    Create lagged features.

    Parameters:
    - df: DataFrame containing time series data with a 'time' column.
    - lag_intervals: List of lag intervals to use for creating lagged features.
    - column: Name of the column to use for creating lagged features.

    Returns:
    - DataFrame with lagged features.
    """

    # Create lagged features
    for lag in lag_intervals:
        df[f'{column}_lag_{lag}'] = df[column].shift(lag)

    return df
lag_intervals = [1, 3]

train_data = lag_feature(train_data, lag_intervals, 'temperature')
train_data = lag_feature(train_data, lag_intervals, 'relative_humidity')

test_data = lag_feature(test_data, lag_intervals, 'temperature')
test_data = lag_feature(test_data, lag_intervals, 'relative_humidity')

In [ ]:
train_data['day_of_week'] = train_data['time'].dt.dayofweek
train_data['hour_of_day'] = train_data['time'].dt.hour

test_data['day_of_week'] = test_data['time'].dt.dayofweek
test_data['hour_of_day'] = test_data['time'].dt.hour

In [ ]:
train_data.head()

In [ ]:
train_data.isna().sum()

In [ ]:
# Replace Nan values with 0 for the lagged features since they are the first values in the time series
train_data['temperature_lag_1'].fillna(0, inplace=True)
train_data['temperature_lag_3'].fillna(0, inplace=True)
train_data['relative_humidity_lag_1'].fillna(0, inplace=True)
train_data['relative_humidity_lag_3'].fillna(0, inplace=True)

test_data['temperature_lag_1'].fillna(0, inplace=True)
test_data['temperature_lag_3'].fillna(0, inplace=True)
test_data['relative_humidity_lag_1'].fillna(0, inplace=True)
test_data['relative_humidity_lag_3'].fillna(0, inplace=True)

train_data.isna().sum()

In [ ]:
def standardize_numeric_columns(df):
    """
    Standardize all numeric columns in the DataFrame.

    Parameters:
    - df: DataFrame containing time series data.

    Returns:
    - DataFrame with numeric columns (except 'time') standardized.
    """
    numeric_columns = df.select_dtypes(include=['number']).columns
    for column in numeric_columns:
        if column != 'time':
            mean = df[column].mean()
            std = df[column].std()
            df[column] = (df[column] - mean) / std
    return df

train_data = standardize_numeric_columns(train_data)
test_data = standardize_numeric_columns(test_data)

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

In [ ]:
features = [
    'temperature', 'dew_point', 'wind_speed', 'wind_direction', 'visibility', 'clouds.total_cover',
    'relative_humidity', 'temperature_lag_1', 'temperature_lag_3',
    'relative_humidity_lag_1', 'relative_humidity_lag_3',
    'day_of_week', 'hour_of_day'
]

X_train = train_data[features]
y_train = train_data['temperature']  

X_test = test_data[features]
y_test = test_data['temperature']  

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


params = {
    'objective': 'regression',
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


num_round = 100  # Number of boosting rounds (iterations)
model = lgb.train(params, train_data, num_round)


y_pred = model.predict(X_test, num_iteration=model.best_iteration)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")

In [ ]:
X_train = torch.tensor(X_train.values, dtype=torch.float32).to(DEVICE)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(DEVICE)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(DEVICE)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(DEVICE)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
class TimeSeriesChunkDataset(Dataset):
    def __init__(self, data, targets, sequence_length):
        self.data = data
        self.targets = targets
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length + 1

    def __getitem__(self, idx):
        return (
            self.data[idx:idx+self.sequence_length],
            self.targets[idx+self.sequence_length-1]
        )

In [ ]:
sequence_length = 1  # Adjust this based on your needs
batch_size = 32

# Create instances of your custom dataset for training and testing
train_dataset = TimeSeriesChunkDataset(X_train, y_train, sequence_length)
test_dataset = TimeSeriesChunkDataset(X_test, y_test, sequence_length)


In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True  # Optional: Drop the last batch if it's smaller than batch_size
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc_1 = nn.Linear(hidden_size, 128)
        self.fc = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(DEVICE)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(DEVICE)

        # Propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        hn = hn.view(-1, self.hidden_size)
        out = self.relu(hn)
        out = self.fc_1(out)
        out = self.relu(out)
        out = self.fc(out)

        return out

In [ ]:
input_size = 13
hidden_size = 64
num_layers = 32
num_classes = 1
learning_rate = 1e-5

In [ ]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, sequence_length).to(DEVICE)


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = lstm(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')